In [ ]:
from depmapomics import tracker 
import pandas as pd
from genepy.utils import helper as h

In [ ]:
t = tracker.getTracker()

In [ ]:
for a in t.arxspan_id.unique():
    entries = t[t.arxspan_id == a]
    if "SANGER" in entries.source.tolist():
        sangerid = "MC-" + a[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True)
        sangeridx = t[(t.source == "SANGER") & (t.arxspan_id == a)].index
        t.loc[sangeridx, "model_condition_id"] = sangerid
    broadidx = t[(~t.source.isin(["SANGER"])) & (t.arxspan_id == a)].index
    broadid = "MC-" + a[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True)
    t.loc[broadidx, "model_condition_id"] = broadid

In [ ]:
t[t.model_condition_id.isnull()]

In [ ]:
check = [x[-4:] for x in t.model_condition_id.unique().tolist()]
len(check)

In [ ]:
len(set(check))

In [ ]:
from genepy.google.google_sheet import dfToSheet

dfToSheet(t, "ccle sample tracker - 22Q2", secret="../../.credentials.json")

In [ ]:
# some 21Q3 samples are missing source annotation
# adding it back here
t = tracker.getTracker()
missing_annot = t[t.source.isnull()]

In [ ]:
import dalmatian as dm

rna_depmap = dm.WorkspaceManager("terra-broad-cancer-prod/CCLE_DepMap_RNAseq").disable_hound().get_samples()
rna_ibm = dm.WorkspaceManager("terra-broad-cancer-prod/Getz_IBM_CellLines_RNASeqData").disable_hound().get_samples()
wgs_depmap = dm.WorkspaceManager("terra-broad-cancer-prod/DepMap_WGS").disable_hound().get_samples()
wgs_ibm = dm.WorkspaceManager("terra-broad-cancer-prod/Getz_IBM_CellLines_WGS").disable_hound().get_samples()

In [ ]:
rna_missing = missing_annot[missing_annot.datatype == 'rna']
for i in rna_missing.index:
    arx = rna_missing.loc[i, 'arxspan_id']
    if rna_depmap.individual_alias.str.startswith(arx).sum() > 0 and rna_ibm.individual_alias.str.startswith(arx).sum() > 0:
        print(i)
    if rna_depmap.individual_alias.str.startswith(arx).sum() > 0:
        t.loc[i, 'source'] = 'DEPMAP'
    elif rna_ibm.individual_alias.str.startswith(arx).sum() > 0:
        t.loc[i, 'source'] = 'IBM'

In [ ]:
wgs_missing = missing_annot[missing_annot.datatype == 'wgs']
for i in wgs_missing.index:
    arx = wgs_missing.loc[i, 'arxspan_id']
    if wgs_depmap.individual_alias.str.startswith(arx).sum() > 0 and wgs_ibm.individual_alias.str.startswith(arx).sum() > 0:
        print(i)
    if wgs_depmap.individual_alias.str.startswith(arx).sum() > 0:
        t.loc[i, 'source'] = 'DEPMAP'
    elif wgs_ibm.individual_alias.str.startswith(arx).sum() > 0:
        t.loc[i, 'source'] = 'IBM'

In [ ]:
# updating sources
from genepy.google.google_sheet import dfToSheet

dfToSheet(t, "ccle sample tracker - 22Q2", secret="../../.credentials.json")

In [ ]:
# generate profile ids
t = tracker.getTracker()
for mc in t.model_condition_id.unique():
    entries = t[t.model_condition_id == mc]
    for d in entries.datatype.unique():
        for s in entries.source.unique():
            for b in t[(t.model_condition_id == mc) & (t.datatype == d) & (t.source == s)].baits.unique():
                idx = t[(t.model_condition_id == mc) & (t.datatype == d) & (t.baits == b) & (t.source == s)].index
                t.loc[idx, 'profile_id'] = "PR-" + h.randomString(stringLength=6, stype="all", withdigits=True)

In [ ]:
t[t.profile_id.isnull()]

In [ ]:
len((t['model_condition_id'].astype(str) + t['datatype'].astype(str) + t['source'].astype(str) + t['baits'].astype(str)).unique())

In [ ]:
# making sure there's no collision
len(t.profile_id.unique())

In [ ]:
dfToSheet(t, "ccle sample tracker - 22Q2", secret="../../.credentials.json")

In [ ]:
t = tracker.getTracker()
len(t.profile_id.unique())

In [ ]:
# example to demonstrate the new system
t[t.arxspan_id == 'ACH-000833'][['model_condition_id', 'profile_id', 'source', 'datatype', 'version']]

In [ ]:
# backfill for legacy data
from taigapy import TaigaClient
tc = TaigaClient()
legacy_hybridcapture = tc.get(name='mutations-da6a', version=9, file='legacy_hybridcapture')
legacy_raindance = tc.get(name='mutations-da6a', version=9, file='legacy_raindance')
legacy_rna = tc.get(name='mutations-da6a', version=9, file='legacy_rna')
legacy_wes_sanger = tc.get(name='mutations-da6a', version=9, file='legacy_wes_sanger')
legacy_wgs_exoniconly = tc.get(name='mutations-da6a', version=9, file='legacy_wgs_exoniconly')

In [ ]:
legacy_hybridcapture.columns

In [ ]:
# hybrid capture: no missing data on the tracker, no need to invent new ones for legacy
hc_tracker= t[t.datatype == 'hybrid_capture']
sum(~legacy_hybridcapture.DepMap_ID.isin(hc_tracker.arxspan_id.unique()))

In [ ]:
# raindance: no missing data on the tracker, no need to invent new ones for legacy
rd_tracker= t[t.datatype == 'raindance']
sum(~legacy_raindance.DepMap_ID.isin(rd_tracker.arxspan_id.unique()))

In [ ]:
# rna: no missing data on the tracker, no need to invent new ones for legacy
rna_tracker= t[t.datatype == 'rna']
sum(~legacy_rna.DepMap_ID.isin(rna_tracker.arxspan_id.unique()))

In [ ]:
# sanger wes: some lines we don't have bams for and are not on the tracker,
# need to invent MC and PR ids for these lines
sanger_wes_tracker = t[(t.datatype == 'wes') & (t.source == 'SANGER')]
missing_sanger_wes = legacy_wes_sanger[~legacy_wes_sanger.DepMap_ID.isin(sanger_wes_tracker.arxspan_id.unique())].DepMap_ID.unique()
has_depmap_wes = []
no_depmap_wes = []
for m in missing_sanger_wes:
    if not len(t[(t.arxspan_id == m) & (t.datatype == 'wes')]) > 0:
        no_depmap_wes.append(m)
    elif len(t[(t.arxspan_id == m) & (t.datatype == 'wes') & (t.source != "SANGER")]) > 0:
        #print(t[(t.arxspan_id == m) & (t.datatype == 'wes') & (t.source != "SANGER")].source[0])
        has_depmap_wes.append(m)
missing_sanger_wes

In [ ]:
# create MC and PR ids for the lines that are missing on the tracker
# arxspan : MC-id : PR-id is 1:1:1 here
for a in missing_sanger_wes:
    print("MC-" + a[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True))

In [ ]:
for a in missing_sanger_wes:
    print("PR-" + h.randomString(stringLength=6, stype="all", withdigits=True))

In [ ]:
# wgs: no missing data on the tracker, no need to invent new ones for legacy
wgs_tracker = t[t.datatype == 'wgs']
legacy_wgs_exoniconly[~legacy_wgs_exoniconly.DepMap_ID.isin(wgs_tracker.arxspan_id.unique())].DepMap_ID.unique()

In [ ]:
# segments
t = tracker.getTracker()

from taigapy import TaigaClient
tc = TaigaClient()
legacy_segments = tc.get(name='copy-number-5f61', version=4, file='legacy_segments')
legacy_segments.Source.unique()

In [ ]:
from gsheets import Sheets
from depmapomics.config import *
backfilled_t = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0)
backfilled_t

In [ ]:
# since some sanger wes lines have already been added with mutation legacy
# only added ACH-002378 which doesn't have mutation data
sanger_wes_seg = legacy_segments[legacy_segments.Source == 'Sanger WES']
sanger_wes_tracker = t[(t.datatype == 'wes') & (t.source == 'SANGER')]
sanger_wes_seg[~sanger_wes_seg.DepMap_ID.isin(sanger_wes_tracker.arxspan_id.unique())].DepMap_ID.unique()


In [ ]:
# ACH-003000 was a fake arxspan id, now it's been renamed to ACH-003020
# TODO: rename this for legacy
# ACH-001650 is missing from the tracker, assuming we have no bam for it, created PR-id (but used existing MC-id)
broad_wes_seg = legacy_segments[legacy_segments.Source == 'Broad WES']
broad_wes_tracker = t[(t.datatype == 'wes') & (t.source != 'SANGER')]
broad_wes_seg[~broad_wes_seg.DepMap_ID.isin(broad_wes_tracker.arxspan_id.unique())].DepMap_ID.unique()

In [ ]:
# each SNP array line needs a new PR id since we don't have SNP data on the tracker at all
# but some of them don't need a new MC id, since the same line has other datatypes on the tracker already
broad_snp_seg = legacy_segments[legacy_segments.Source == 'Broad SNP']
broad_snp_tracker = t[(t.source != 'SANGER')]
missing_broad_snp = broad_snp_seg[~broad_snp_seg.DepMap_ID.isin(broad_snp_tracker.arxspan_id.unique())].DepMap_ID.unique()
#len(broad_snp_seg.DepMap_ID.unique())

In [ ]:
snp_has_mc = broad_snp_seg[broad_snp_seg.DepMap_ID.isin(broad_snp_tracker.arxspan_id.unique())].DepMap_ID.unique()
# for the lines that have MC ids on the tracker, retrieve them here
for i in snp_has_mc:
    print(t[(t.arxspan_id == i) & (t.source != 'SANGER')].model_condition_id.unique()[0])

In [ ]:
# for the lines that don't have MC ids on the tracker, generate new ones
for i in missing_broad_snp:
    print("MC-" + i[4:] + "-" + h.randomString(stringLength=4, stype="all", withdigits=True))

In [ ]:
# generate PR ids for all snp lines
for i in broad_snp_seg.DepMap_ID.unique():
    print("PR-" + h.randomString(stringLength=6, stype="all", withdigits=True))

In [ ]:
# one chordoma line doesn't have a chordoma wes on tracker, but has a sanger wes
# not creating PR/MC id for this because it will not get used
chordoma_seg = legacy_segments[legacy_segments.Source == 'Chordoma WES']
chordoma_tracker = t[(t.datatype == 'wes') & (t.source == 'CHORDOMA')]
chordoma_seg[~chordoma_seg.DepMap_ID.isin(chordoma_tracker.arxspan_id.unique())].DepMap_ID.unique()
#chordoma_seg.DepMap_ID.unique()

In [ ]:
print("MC-" + '002204' + "-" + h.randomString(stringLength=4, stype="all", withdigits=True))

## Now all lines have been backfilled with MC- and PR-id. The following section populates the modelCondition tab on "New Data transfer proposal"
ALL model conditions will be recorded in the modelCondition tab, no matter if they are blacklisted or if they're only present in legacy. Blacklist info will be on the omicsProfile tab

In [ ]:
t = tracker.getTracker()
backfilled_t = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[1].to_frame(index_col=0)


In [ ]:
len(set([x[10:] for x in t.model_condition_id.astype(str)]))

In [ ]:
len(set([x[10:] for x in backfilled_t.model_condition_id.astype(str)]))

In [ ]:
len(set([x[10:] for x in t.model_condition_id.astype(str)] + [x[10:] for x in backfilled_t.model_condition_id.astype(str)]))

In [ ]:
-124 + 2725 + 171

In [ ]:
len(set(t.profile_id.astype(str)))

In [ ]:
len(set(backfilled_t.index.astype(str)))

In [ ]:
len(set(t.profile_id.tolist() + backfilled_t.index.tolist()))

In [ ]:
mc_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[].to_frame(index_col=0)

In [ ]:
mc_tab

In [ ]:
for i in list(set(t.model_condition_id.tolist() + backfilled_t.model_condition_id.tolist())):
    print(i)

In [ ]:
all_mcs = t.model_condition_id.unique().tolist() + backfilled_t.model_condition_id.unique().tolist()

In [ ]:
import json

rna_renaming = {}

with open('../temp/22Q2/rna_sample_renaming.json') as json_file:
    rna_renaming = json.load(json_file)

In [ ]:
mc_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0)
mc_tab

In [ ]:
# backfill expression and fusion on the model condition tab (no legacy)
t = tracker.getTracker()

for k in rna_renaming:
    if k != 'transcript_id(s)':
        mc = t.loc[k, 'model_condition_id']
        pr = t.loc[k, 'profile_id']
        mc_tab.loc[mc, 'expression'] = pr
        mc_tab.loc[mc, 'fusion'] = pr
        

In [ ]:
import pygsheets

gc = pygsheets.authorize(service_file="../../.credentials.json")

In [ ]:
sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','ModelCondition')
# wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
from gsheets import Sheets
from depmapomics.config import *

mc_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0)
mc_tab

In [ ]:
# fill in source column on the model condition tab
# only 2 categories for this tab: sanger vs broad
source_dict = pd.concat([t[['model_condition_id', 'source']], legacy_tab[['model_condition_id', 'source']]], ignore_index=True)
for i in mc_tab.index:
    if 'SANGER' in source_dict[source_dict.model_condition_id == i].source.unique().tolist() or 'SANGER_NOBAM' in source_dict[source_dict.model_condition_id == i].source.unique().tolist():
        mc_tab.loc[i, 'source'] = 'SANGER'
    else:
        mc_tab.loc[i, 'source'] = 'BROAD'
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
# backfill rna mut legacy on the model condition tab
for i in legacy_rna.DepMap_ID.unique():
    if len(t[(t.arxspan_id == i) & (t.datatype == 'rna')].model_condition_id.unique()) == 1:
        mc = t[(t.arxspan_id == i) & (t.datatype == 'rna')].model_condition_id.unique()[0]
        pr = t[(t.arxspan_id == i) & (t.datatype == 'rna') & (t.source == 'CCLE2')].profile_id.unique()[0]
        mc_tab.loc[mc, 'mut_rna'] = pr
        if len(t[(t.arxspan_id == i) & (t.datatype == 'rna') & (t.source == 'CCLE2')].profile_id.unique()) != 1:
            print(i)
    else:
        print('multiple mc-ids corresponding to one arxspan: ', i)
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
# backfill hc mut legacy on the model condition tab
for i in legacy_hybridcapture.DepMap_ID.unique():
    if len(t[(t.arxspan_id == i) & (t.datatype == 'hybrid_capture')].model_condition_id.unique()) == 1:
        mc = t[(t.arxspan_id == i) & (t.datatype == 'hybrid_capture')].model_condition_id.unique()[0]
        pr = t[(t.arxspan_id == i) & (t.datatype == 'hybrid_capture') & (t.source == 'CCLE2')].profile_id.unique()[0]
        mc_tab.loc[mc, 'mut_hc'] = pr
        if len(t[(t.arxspan_id == i) & (t.datatype == 'hybrid_capture') & (t.source == 'CCLE2')].profile_id.unique()) != 1:
            print(i)
    else:
        print('multiple mc-ids corresponding to one arxspan: ', i)
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
# backfill rd mut legacy on the model condition tab
for i in legacy_raindance.DepMap_ID.unique():
    if len(t[(t.arxspan_id == i) & (t.datatype == 'raindance')].model_condition_id.unique()) == 1:
        mc = t[(t.arxspan_id == i) & (t.datatype == 'raindance')].model_condition_id.unique()[0]
        pr = t[(t.arxspan_id == i) & (t.datatype == 'raindance') & (t.source == 'CCLE2')].profile_id.unique()[0]
        mc_tab.loc[mc, 'mut_rd'] = pr
        if len(t[(t.arxspan_id == i) & (t.datatype == 'raindance') & (t.source == 'CCLE2')].profile_id.unique()) != 1:
            print(i)
    else:
        print('multiple mc-ids corresponding to one arxspan: ', i)
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
mc_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[1].to_frame(index_col=0)
legacy_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[2].to_frame(index_col=0)

In [ ]:
# backfill sanger wes mut legacy on the model condition tab
for i in legacy_wes_sanger.DepMap_ID.unique():
    mc_tracker = t[(t.arxspan_id == i) & (t.datatype == 'wes') & (t.source == 'SANGER')].model_condition_id.unique().tolist()
    mc_legacy = legacy_tab[(legacy_tab.arxspan_id == i) & (legacy_tab.source == 'SANGER_NOBAM') & (legacy_tab.datatype == 'wes')].model_condition_id.unique().tolist()
    if len(set(mc_tracker + mc_legacy)) > 1:
        print('more than 1 mc-ids corresponding to one arxspan: ', i)
    mc = list(set(mc_tracker + mc_legacy))[0]
    if mc in mc_tracker:
        pr = t[(t.model_condition_id == mc) & (t.source == 'SANGER') & (t.datatype == 'wes')].profile_id.unique().tolist()
        if len(pr) > 1:
            print('more than 1 pr-ids corresponding to one mc-id: ', mc)
        mc_tab.loc[mc, 'mut_sanger_wes'] = pr
    else:
        pr = legacy_tab[(legacy_tab.model_condition_id == mc) & (legacy_tab.source == 'SANGER_NOBAM') & (legacy_tab.datatype == 'wes')].index.unique().tolist()
        if len(pr) > 1:
            print('more than 1 pr-ids corresponding to one mc-id: ', mc)
        mc_tab.loc[mc, 'mut_sanger_wes'] = pr

In [ ]:
# backfill wgs mut legacy on the model condition tab
for i in legacy_wgs_exoniconly.DepMap_ID.unique():
    if len(t[(t.arxspan_id == i) & (t.datatype == 'wgs')].model_condition_id.unique()) == 1:
        mc = t[(t.arxspan_id == i) & (t.datatype == 'wgs')].model_condition_id.unique()[0]
        pr = t[(t.arxspan_id == i) & (t.datatype == 'wgs') & (t.source == 'CCLE2')].profile_id.unique()[0]
        mc_tab.loc[mc, 'mut_wgs'] = pr
        if len(t[(t.arxspan_id == i) & (t.datatype == 'wgs') & (t.source == 'CCLE2')].profile_id.unique()) != 1:
            print(i)
    else:
        print('multiple mc-ids corresponding to one arxspan: ', i)
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
# backfill
import json

wgs_renaming = {}
wes_renaming = {}

with open('../temp/22Q2/wgs_sample_renaming.json') as json_file:
    wgs_renaming = json.load(json_file)

with open('../temp/22Q2/wes_sample_renaming.json') as json_file:
    wes_renaming = json.load(json_file)


In [ ]:
# backfill cga wes mut
# for the same MC, prioritize wes mutation
from depmapomics.config import *

# some lines failed CN QC, but we have been rescuing them for mutation data.
# these lines aren't saved in the renaming json files as of 22Q2, so add them here
wes_renaming.update(RESCUE_FOR_MUTATION_WES)
wgs_renaming.update(RESCUE_FOR_MUTATION_WGS)

mcs = set([])
for ke in wes_renaming:
    mc = t.loc[ke, 'model_condition_id']
    pr = t.loc[ke, 'profile_id']
    mcs.add(mc)
    mc_tab.loc[mc, 'mut_cga_wes'] = pr
for kw in wgs_renaming:
    mc = t.loc[kw, 'model_condition_id']
    if mc not in mcs:
        pr = t.loc[kw, 'profile_id']
        mcs.add(mc)
        mc_tab.loc[mc, 'mut_cga_wes'] = pr
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')

In [ ]:
# backfill germline matrix
# here use the renaming dicts before appending rescued lines
wgs_mat = pd.read_csv("../temp/binary_mutguides_wgs.tsv.gz", sep='\t', index_col='Unnamed: 0')
wes_mat = pd.read_csv("../temp/binary_mutguides_wes.tsv.gz", sep='\t', index_col='Unnamed: 0')

wgs_mat_noguides = wgs_mat.iloc[:, 4:]
wes_mat_noguides = wes_mat.iloc[:, 4:]

wgs_whitelist = [x for x in wgs_mat_noguides.columns if x in wgs_renaming]
wes_whitelist = [x for x in wes_mat_noguides.columns if x in wes_renaming]

In [ ]:
# use the same prioritization scheme as make_germline_matrix_wgs.ipynb
mcs = set([])
for kw in wgs_whitelist:
    mc = t.loc[kw, 'model_condition_id']
    pr = t.loc[kw, 'profile_id']
    mcs.add(mc)
    mc_tab.loc[mc, 'germline_matrix'] = pr
for ke in wes_whitelist:
    mc = t.loc[ke, 'model_condition_id']
    if mc not in mcs:
        pr = t.loc[ke, 'profile_id']
        mcs.add(mc)
        mc_tab.loc[mc, 'germline_matrix'] = pr
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')    

In [ ]:
# backfill cnv, using the same local jsons without rescued lines
mcs = set([])
for kw in wgs_renaming:
    mc = t.loc[kw, 'model_condition_id']
    pr = t.loc[kw, 'profile_id']
    mcs.add(mc)
    mc_tab.loc[mc, 'cnv'] = pr
for ke in wes_renaming:
    mc = t.loc[ke, 'model_condition_id']
    if mc not in mcs:
        pr = t.loc[ke, 'profile_id']
        mcs.add(mc)
        mc_tab.loc[mc, 'cnv'] = pr
wksht.set_dataframe(mc_tab, 'A1', copy_index=True, nan='')    

# Backfill the omicProfile tab

In [ ]:
from gsheets import Sheets
from depmapomics.config import *

pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=2)
#t = tracker.getTracker()
pr_tab

In [ ]:
portal_cols = {'public': 'public_release_date', 'dmc': 'consortium_release_date', 'internal': 'internal_release_date', 'ibm': 'ibm_release_date'}
portal_taiga = {'internal': 'internal-22q2-097a', 'ibm': 'ibm-22q2-a71e', 'dmc': 'dmc-22q2-5e51', 'public': 'public-22q2-de04'}

In [ ]:
# backfill rna profiles
import json
from taigapy import TaigaClient
tc = TaigaClient()

rna_renaming = {}
with open('../temp/22Q2/rna_sample_renaming.json') as json_file:
    rna_renaming = json.load(json_file)
rna_renaming_swapped = dict([(value, key) for key, value in rna_renaming.items()])

In [ ]:
rna_renaming_swapped['ACH-000008']

In [ ]:
for i in portal_taiga:
    CCLE_expression = tc.get(name=portal_taiga[i], file='CCLE_expression')
    lines = CCLE_expression.index.tolist()
    for l in lines:
        cds_id = rna_renaming_swapped[l]
        pr = t.loc[cds_id, 'ProfileID']
        mc = t.loc[cds_id, 'ModelConditionID']
        pr_tab.loc[pr, 'ModelCondition'] = mc
        pr_tab.loc[pr, 'Datatype'] = 'rna'
        pr_tab.loc[pr, 'Baits'] = t.loc[cds_id, 'baits']
        pr_tab.loc[pr, 'BlacklistOmics'] = t.loc[cds_id, 'blacklist']
        pr_tab.loc[pr, 'Issue'] = t.loc[cds_id, 'issue']
        pr_tab.loc[pr, 'Prioritized'] = t.loc[cds_id, 'prioritized']
        pr_tab.loc[pr, 'CDSID'] = cds_id
        pr_tab.loc[pr, 'ProfileSource'] = 'bam'
        pr_tab.loc[pr, portal_cols[i]] = '22Q2'

In [ ]:
pr_tab.loc['PR-bKcagB', 'CDSID']

In [ ]:
sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')

wksht.set_dataframe(pr_tab, 'A3', copy_index=True, nan='') 

In [ ]:
# backfill WGS/WES profiles
wgs_renaming = {}
wes_renaming = {}

with open('../temp/22Q2/wgs_sample_renaming.json') as json_file:
    wgs_renaming = json.load(json_file)

with open('../temp/22Q2/wes_sample_renaming.json') as json_file:
    wes_renaming = json.load(json_file)

In [ ]:
# combine renaming dictionaries, prioritizing WES
wes_renaming.update(RESCUE_FOR_MUTATION_WES)
wgs_renaming.update(RESCUE_FOR_MUTATION_WGS)
wgs_renaming_swapped = dict([(value, key) for key, value in wgs_renaming.items()])
wes_renaming_swapped = dict([(value, key) for key, value in wes_renaming.items()])

combined_mut = wes_renaming_swapped.copy()
for i in wgs_renaming_swapped:
    if i not in combined_mut:
        combined_mut[i] = wgs_renaming_swapped[i]

In [ ]:
# backfill WES_CGA mut
pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=1)

for i in portal_taiga:
    CCLE_mutations = tc.get(name=portal_taiga[i], file='CCLE_mutations')
    AC_cols = ["CGA_WES_AC", "HC_AC", "RD_AC", "RNAseq_AC", "SangerWES_AC", "WGS_AC"]
    mut_notnull1 = CCLE_mutations.groupby("DepMap_ID")[AC_cols].apply(lambda x: x.notnull().any())
    lines = mut_notnull1.index.tolist()
    for l in lines:
        if mut_notnull1.loc[l, 'CGA_WES_AC'] == True:
            cds_id = combined_mut[l]
            pr = t.loc[cds_id, 'profile_id']
            mc = t.loc[cds_id, 'model_condition_id']
            pr_tab.loc[pr, 'ModelCondition'] = mc
            pr_tab.loc[pr, 'datatype'] = t.loc[cds_id, 'datatype']
            pr_tab.loc[pr, 'baits'] = t.loc[cds_id, 'baits']
            pr_tab.loc[pr, 'blacklist_omics'] = t.loc[cds_id, 'blacklist']
            pr_tab.loc[pr, 'issue'] = t.loc[cds_id, 'issue']
            pr_tab.loc[pr, 'prioritized'] = t.loc[cds_id, 'prioritized']
            pr_tab.loc[pr, 'cds_id'] = cds_id
            pr_tab.loc[pr, 'profile_source'] = 'BAM'
            pr_tab.loc[pr, portal_cols[i]] = '22Q2'            

In [ ]:
sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')

wksht.set_dataframe(pr_tab, 'A2', copy_index=True, nan='') 

In [ ]:
# backfill legacy mut (everything except sanger wes)
# the rna are overwriting the expression/fusion profiles but that's ok ?
pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=1)

legacy_mapping = {"HC_AC": "hybrid_capture", "RD_AC": "raindance", "RNAseq_AC": "rna", "WGS_AC": "wgs"}

for i in portal_taiga:
    CCLE_mutations = tc.get(name=portal_taiga[i], file='CCLE_mutations')
    AC_cols = ["CGA_WES_AC", "HC_AC", "RD_AC", "RNAseq_AC", "SangerWES_AC", "WGS_AC"]
    mut_notnull1 = CCLE_mutations.groupby("DepMap_ID")[AC_cols].apply(lambda x: x.notnull().any())
    lines = mut_notnull1.index.tolist()
    for l in lines:
        for k in legacy_mapping:
            if mut_notnull1.loc[l, k] == True:
                cds_id = t[(t.arxspan_id == l) & (t.datatype == legacy_mapping[k]) & (t.source == 'CCLE2')].index
                pr = t.loc[cds_id, 'profile_id'].unique()
                mc = t.loc[cds_id, 'model_condition_id'].unique()
                if len(pr) != 1 or len(mc) != 1:
                    print(k, cds_id, 'multiple mc or pr mapped to the same line')
                pr = pr[0]
                mc = mc[0]
                cds_id = cds_id[0]
                pr_tab.loc[pr, 'ModelCondition'] = mc
                pr_tab.loc[pr, 'datatype'] = t.loc[cds_id, 'datatype']
                pr_tab.loc[pr, 'baits'] = t.loc[cds_id, 'baits']
                pr_tab.loc[pr, 'blacklist_omics'] = t.loc[cds_id, 'blacklist']
                pr_tab.loc[pr, 'issue'] = t.loc[cds_id, 'issue']
                pr_tab.loc[pr, 'prioritized'] = t.loc[cds_id, 'prioritized']
                pr_tab.loc[pr, 'cds_id'] = cds_id
                pr_tab.loc[pr, 'profile_source'] = 'mutations-da6a'
                pr_tab.loc[pr, portal_cols[i]] = '22Q2'

sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')
wksht.set_dataframe(pr_tab, 'A2', copy_index=True, nan='') 

In [ ]:
# backfill sanger wes legacy mut
pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=1)
legacy_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[2].to_frame(index_col=0)

for i in portal_taiga:
    CCLE_mutations = tc.get(name=portal_taiga[i], file='CCLE_mutations')
    AC_cols = ["CGA_WES_AC", "HC_AC", "RD_AC", "RNAseq_AC", "SangerWES_AC", "WGS_AC"]
    mut_notnull1 = CCLE_mutations.groupby("DepMap_ID")[AC_cols].apply(lambda x: x.notnull().any())
    lines = mut_notnull1.index.tolist()
    for l in lines:
        if mut_notnull1.loc[l, 'SangerWES_AC'] == True:
            cds_id = t[(t.arxspan_id == l) & (t.datatype == 'wes') & (t.source == 'SANGER')].index
            pr, mc = '', ''
            # if can't find a cds id on the tracker, check the backfilled legacy tab instead
            if len(cds_id) == 0:
                pr = legacy_tab[(legacy_tab.arxspan_id == l) & (legacy_tab.datatype == 'wes') & (legacy_tab.source == 'SANGER_NOBAM') & (legacy_tab.legacy_type == 'mut')].index.tolist()
                mc = legacy_tab[(legacy_tab.arxspan_id == l) & (legacy_tab.datatype == 'wes') & (legacy_tab.source == 'SANGER_NOBAM') & (legacy_tab.legacy_type == 'mut')].model_condition_id.tolist()
                cds_id = []
            else:
                pr = t.loc[cds_id, 'profile_id'].unique().tolist()
                mc = t.loc[cds_id, 'model_condition_id'].unique().tolist()
            if len(pr) != 1 or len(mc) != 1:
                print(k, cds_id, 'multiple mc or pr mapped to the same line')
            pr = pr[0]
            mc = mc[0]
            pr_tab.loc[pr, 'ModelCondition'] = mc
            pr_tab.loc[pr, 'datatype'] = 'wes'
            pr_tab.loc[pr, 'profile_source'] = 'mutations-da6a'
            pr_tab.loc[pr, portal_cols[i]] = '22Q2'
            if len(cds_id) > 0:
                cds_id = cds_id[0]
                pr_tab.loc[pr, 'baits'] = t.loc[cds_id, 'baits']
                pr_tab.loc[pr, 'blacklist_omics'] = t.loc[cds_id, 'blacklist']
                pr_tab.loc[pr, 'issue'] = t.loc[cds_id, 'issue']
                pr_tab.loc[pr, 'prioritized'] = t.loc[cds_id, 'prioritized']
                pr_tab.loc[pr, 'cds_id'] = cds_id
                
sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')
wksht.set_dataframe(pr_tab, 'A2', copy_index=True, nan='') 

In [ ]:
# backfill CN
wgs_renaming = {}
wes_renaming = {}

with open('../temp/22Q2/wgs_sample_renaming.json') as json_file:
    wgs_renaming = json.load(json_file)

with open('../temp/22Q2/wes_sample_renaming.json') as json_file:
    wes_renaming = json.load(json_file)

wgs_renaming_swapped = dict([(value, key) for key, value in wgs_renaming.items()])
wes_renaming_swapped = dict([(value, key) for key, value in wes_renaming.items()])

combined_cn = wgs_renaming_swapped.copy()
for i in wes_renaming_swapped:
    if i not in combined_cn:
        combined_cn[i] = wes_renaming_swapped[i]

In [ ]:
pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=1)
legacy_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[2].to_frame(index_col=0)

legacy_seg = {'Sanger WES': ['SANGER', 'wes'], 'Broad SNP': ['CCLE2', 'SNParray'], 'Broad WES': ['Broad', 'wes'], 'Chordoma WES': ['CHORDOMA', 'wes']}

for i in portal_taiga:
    CCLE_segment_cn = tc.get(name=portal_taiga[i], file='CCLE_segment_cn')
    lines = CCLE_segment_cn.DepMap_ID.unique().tolist()
    for l in lines:
        if l in combined_cn:
            cds_id = combined_cn[l]
            pr = t.loc[cds_id, 'profile_id']
            mc = t.loc[cds_id, 'model_condition_id']
            pr_tab.loc[pr, 'ModelCondition'] = mc
            pr_tab.loc[pr, 'datatype'] = t.loc[cds_id, 'datatype']
            pr_tab.loc[pr, 'baits'] = t.loc[cds_id, 'baits']
            pr_tab.loc[pr, 'blacklist_omics'] = t.loc[cds_id, 'blacklist']
            pr_tab.loc[pr, 'issue'] = t.loc[cds_id, 'issue']
            pr_tab.loc[pr, 'prioritized'] = t.loc[cds_id, 'prioritized']
            pr_tab.loc[pr, 'cds_id'] = cds_id
            pr_tab.loc[pr, 'profile_source'] = 'BAM'
            pr_tab.loc[pr, portal_cols[i]] = '22Q2'
        else:
            seg_source = CCLE_segment_cn[CCLE_segment_cn.DepMap_ID == l].Source.tolist()[0]
            cds_id = t[(t.arxspan_id == l) & (t.source == legacy_seg[seg_source][0]) & (t.datatype == legacy_seg[seg_source][1])].index
            pr, mc = '', ''
            # if can't find a cds id on the tracker, check the backfilled legacy tab instead
            if len(cds_id) == 0:
                pr = legacy_tab[(legacy_tab.arxspan_id == l) & (legacy_tab.datatype == legacy_seg[seg_source][1]) & (legacy_tab.source.str.startswith(legacy_seg[seg_source][0]))].index.tolist()
                mc = legacy_tab[(legacy_tab.arxspan_id == l) & (legacy_tab.datatype == legacy_seg[seg_source][1]) & (legacy_tab.source.str.startswith(legacy_seg[seg_source][0]))].model_condition_id.tolist()
                cds_id = []
            else:
                pr = t.loc[cds_id, 'profile_id'].unique().tolist()
                mc = t.loc[cds_id, 'model_condition_id'].unique().tolist()
            if len(pr) != 1 or len(mc) != 1:
                print(k, cds_id, 'multiple mc or pr mapped to the same line')
            pr = pr[0]
            mc = mc[0]
            pr_tab.loc[pr, 'ModelCondition'] = mc
            pr_tab.loc[pr, 'datatype'] = legacy_seg[seg_source][1]
            pr_tab.loc[pr, 'profile_source'] = 'copy-number-5f61'
            pr_tab.loc[pr, portal_cols[i]] = '22Q2'
            if len(cds_id) > 0:
                cds_id = cds_id[0]
                pr_tab.loc[pr, 'baits'] = t.loc[cds_id, 'baits']
                pr_tab.loc[pr, 'blacklist_omics'] = t.loc[cds_id, 'blacklist']
                pr_tab.loc[pr, 'issue'] = t.loc[cds_id, 'issue']
                pr_tab.loc[pr, 'prioritized'] = t.loc[cds_id, 'prioritized']
                pr_tab.loc[pr, 'cds_id'] = cds_id
        
sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')
wksht.set_dataframe(pr_tab, 'A2', copy_index=True, nan='')

In [ ]:
tc.get(name=portal_taiga['internal'], file='CCLE_gene_cn')

In [ ]:
CCLE_segment_cn = tc.get(name='internal-22q2-097a', version=10, file='CCLE_segment_cn')
CCLE_segment_cn

In [ ]:
for i in CCLE_segment_cn.DepMap_ID.unique().tolist():
    if len(CCLE_segment_cn[CCLE_segment_cn.DepMap_ID == i].Source.unique()) > 1:
        print(i)

In [ ]:
CCLE_segment_cn.Source.unique().tolist()

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
legacy_segments = tc.get(name='copy-number-5f61', version=5, file='legacy_segments')
legacy_segments.Source.unique()

# transfer media annotation columns to MC tab
first check for inconsistencies

In [ ]:
t = tracker.getTracker()
mc_tracker = t[['model_condition_id', 'parent_entity_id', 'doublingt', 'CCLE_doubling_time', 'DMX_doubling_time', 'GPP_cell_doubling_time', 'mut_rate', 'characteristics', 'visual_characteristics', 'growth_pattern', 'comments', 'puromycin_selection_ug_ml', 'observed_cell_representation', 'year_passaging_billed', 'month_passaging_billed', 'observed_infection_rate', 'vol_pooled_virus_for_each_well', 'cas9_activity', 'blasticidin_selection_ug_ml', 'transfected', 'freeze_media', 'condition', 'supplements', 'base_media', 'serums', 'glutamine', 'sodium_pyruv', 'neaa', 'its', 'hepes', 'hydrocortisone', 'estradiol', 'sodium_bicarb', 'pen_strep', 'gm_csf', 'other_media']]

In [ ]:
nodup_mc_tracker = mc_tracker.drop_duplicates(ignore_index=True)
nodup_mc_tracker

In [ ]:
for mc in nodup_mc_tracker.model_condition_id.unique():
    if len(nodup_mc_tracker[nodup_mc_tracker.model_condition_id == mc]) > 1:
        for c in nodup_mc_tracker.columns:
            df = nodup_mc_tracker[nodup_mc_tracker.model_condition_id == mc]
            if len(df[c].dropna().unique()) > 1:
                print(mc, 'conflicting col: ', c)
                print(df[c].dropna().unique())

# Correction: if a profile is on the tracker, the 'ProfileSource' should be 'bam'. Otherwise, it should be 'taiga'

In [ ]:
pr_tab = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/10Lg0xkT5OHLYgJ9VKpkh8VR64TXfxPVJXRVAckU8uBg").sheets[0].to_frame(index_col=0, header=1)
t = tracker.getTracker()

for i in pr_tab.index:
    if i in t.profile_id.tolist():
        pr_tab.loc[i, 'ProfileSource'] = 'bam'
    else:
        pr_tab.loc[i, 'ProfileSource'] = 'taiga'

sheet = gc.open('Backfilled profile IDs')
wksht = sheet.worksheet('title','OmicsProfile')
wksht.set_dataframe(pr_tab, 'A2', copy_index=True, nan='')

In [ ]:
rna_t = t[t.datatype == 'rna']
len(rna_t.arxspan_id.unique())

In [ ]:
len(rna_t.profile_id.unique())

In [ ]:
dna_t = t[t.datatype.isin(['wgs', 'wes'])]
len(dna_t.arxspan_id.unique())

In [ ]:
len(dna_t.profile_id.unique())

# add profile ids to legacy datasets

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
legacy_hybridcapture = tc.get(name='mutations-da6a', version=10, file='legacy_hybridcapture')
legacy_raindance = tc.get(name='mutations-da6a', version=10, file='legacy_raindance')
legacy_rna = tc.get(name='mutations-da6a', version=10, file='legacy_rna')
legacy_wes_sanger = tc.get(name='mutations-da6a', version=10, file='legacy_wes_sanger')
legacy_wgs_exoniconly = tc.get(name='mutations-da6a', version=10, file='legacy_wgs_exoniconly')

In [ ]:
t = tracker.getTracker()
hc_t = t[(t.datatype == 'hybrid_capture') & (t.arxspan_id.isin(legacy_hybridcapture.DepMap_ID))]
hc_dict = dict(zip(hc_t.arxspan_id, hc_t.ProfileID))
assert(set(legacy_hybridcapture.DepMap_ID).issubset(hc_dict.keys()))
legacy_hybridcapture['ProfileID'] = legacy_hybridcapture['DepMap_ID'].map(hc_dict)
legacy_hybridcapture.to_csv('../temp/legacy_hybridcapture.csv', index=False)

In [ ]:
rd_t = t[(t.datatype == 'raindance') & (t.arxspan_id.isin(legacy_raindance.DepMap_ID))]
rd_dict = dict(zip(rd_t.arxspan_id, rd_t.ProfileID))
assert(set(legacy_raindance.DepMap_ID).issubset(rd_dict.keys()))
legacy_raindance['ProfileID'] = legacy_raindance['DepMap_ID'].map(hc_dict)
legacy_raindance.to_csv('../temp/legacy_raindance.csv', index=False)

In [ ]:
legacy_raindance

In [ ]:
tc.update_dataset(changes_description="add ProfileID column",
                dataset_permaname='mutations-da6a',
                upload_files=[
                  {
                      "path": '../temp/legacy_hybridcapture.csv',
                      "format": "TableCSV",
                      "encoding": "utf-8"
                  },
                  {
                      "path": '../temp/legacy_raindance.csv',
                      "format": "TableCSV",
                      "encoding": "utf-8"
                  },
                  {
                      "path": '../temp/legacy_rna.csv',
                      "format": "TableCSV",
                      "encoding": "utf-8"
                  },
                  {
                      "path": '../temp/legacy_wes_sanger.csv',
                      "format": "TableCSV",
                      "encoding": "utf-8"
                  },
                  {
                      "path": '../temp/legacy_wgs_exoniconly.csv',
                      "format": "TableCSV",
                      "encoding": "utf-8"
                  },
                ],
                dataset_description="where the legacy mutation lies")